<a href="https://colab.research.google.com/github/Rishabh5752/Text_Attack_Exp/blob/main/BERT_base_uncased_SST2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install textattack

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.7/418.7 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.1/373.1 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 kB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 89.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 109.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 MB 16.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97

In [2]:
!pip install tensorflow-text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 43.4 MB/s eta 0:00:00


In [3]:
import textattack
import transformers

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-SST-2")
tokenizer = transformers.AutoTokenizer.from_pretrained("textattack/bert-base-uncased-SST-2")
model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(model, tokenizer)

In [7]:
# Construct our four components for `Attack`

from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.search_methods import GreedyWordSwapWIR
from textattack.constraints.semantics import WordEmbeddingDistance
from textattack.transformations import WordSwap

In [8]:
goal_function = textattack.goal_functions.UntargetedClassification(model_wrapper)

constraints = [

    RepeatModification(),
    StopwordModification(),
    WordEmbeddingDistance(min_cos_sim=0.7)

]
from textattack.search_methods import GreedySearch

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:12<00:00, 38.1MB/s]
textattack: Unzipping file /root/.cache/textattack/tmpnmiwhkks.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.


In [9]:
from textattack.search_methods import GreedySearch

In [10]:
transformation = textattack.transformations.word_swaps.word_swap_embedding.WordSwapEmbedding()
search_method = textattack.search_methods.GreedySearch()

In [15]:
#attack = textattack.attack_recipes.deepwordbug_gao_2018.DeepWordBugGao2018.build(model_wrapper)
attack =  textattack.Attack(goal_function, constraints, transformation, search_method)

#datasets.load_dataset('emotion', split =['test'])
#dataset=datasets.load_dataset('sms_spam', split =['test'])
dataset = textattack.datasets.HuggingFaceDataset("sst2", dataset_columns=(["sentence"],'label'))
#attack =  textattack.Attack(goal_function, constraints, transformation, search_method)

  0%|          | 0/3 [00:00<?, ?it/s]

textattack: Loading datasets dataset sst2, split train.


In [16]:
attack_args = textattack.AttackArgs(

    num_examples=100,
    disable_stdout=True

)
attacker = textattack.Attacker(attack, dataset, attack_args)

attacker.attack_dataset()

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  15
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 






  0%|          | 0/100 [00:00<?, ?it/s]


  1%|          | 1/100 [00:05<09:29,  5.75s/it]


[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   1%|          | 1/100 [00:05<09:30,  5.76s/it]


[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   2%|▏         | 2/100 [00:07<06:13,  3.81s/it]


[Succeeded / Failed / Skipped / Total] 1 / 1 / 0 / 2:   2%|▏         | 2/100 [00:07<06:13,  3.82s/it]


[Succeeded / Failed / Skipped / Total] 1 / 1 / 0 / 2:   3%|▎         | 3/100 [00:12<06:49,  4.22s/it]


[Succeeded / Failed / Skipped / Total] 2 / 1 / 0 / 3:   3%|▎         | 3/100 [00:12<06:49,  4.22s/it]


[Succeeded / Failed / Skipped / Total] 2 / 1 / 0 / 3:   4%|▍         | 4/100 [00:14<05:51,  3.66s/it]


[Succeeded / Failed / Skipped / Total] 3 / 1 / 0 / 4:   4%|▍         | 4/100 [00:14<05:51,  3.66s/it]


[Succeeded / Failed / Skipped / Total] 3 / 1 / 0 / 4:   5%|▌         | 5/100 [00:16<05:18,  3.36s/it]


[Succeeded / Failed / Skipped / Total] 3 / 2 / 0 / 5:   5%|▌         | 5/


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 70     |
| Number of failed attacks:     | 29     |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 99.0%  |
| Accuracy under attack:        | 29.0%  |
| Attack success rate:          | 70.71% |
| Average perturbed word %:     | 28.28% |
| Average num. words per input: | 9.01   |
| Avg num queries:              | 95.75  |
+-------------------------------+--------+


In [17]:
from textattack import Attacker

attacker = Attacker(attack, dataset)
attack_results=attacker.attack_dataset()

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  15
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 






  0%|          | 0/10 [00:00<?, ?it/s]


 10%|█         | 1/10 [00:03<00:30,  3.41s/it]

--------------------------------------------- Result 1 ---------------------------------------------


  0%|          | 0/100 [06:34<?, ?it/s]



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  10%|█         | 1/10 [00:04<00:36,  4.09s/it]

[[Negative (99%)]] --> [[Positive (52%)]]

[[hide]] [[new]] secretions from the [[parental]] units 

[[hidden]] [[newest]] secretions from the [[paternal]] units 




...

--------------------------------------------- Result 8 ---------------------------------------------
[[Positive (100%)]] --> [[Negative (100%)]]

of [[saucy]] 

of [[smutty]] 







[Succeeded / Failed / Skipped / Total] 5 / 3 / 0 / 8:  90%|█████████ | 9/10 [00:28<00:03,  3.17s/it]


[Succeeded / Failed / Skipped / Total] 5 / 4 / 0 / 9:  90%|█████████ | 9/10 [00:28<00:03,  3.17s/it]

--------------------------------------------- Result 9 ---------------------------------------------
[[Negative (100%)]] --> [[[FAILED]]]

a depressed fifteen-year-old 's suicidal poetry 







[Succeeded / Failed / Skipped / Total] 5 / 4 / 0 / 9: 100%|██████████| 10/10 [00:30<00:00,  3.01s/it]


[Succeeded / Failed / Skipped / Total] 6 / 4 / 0 / 10: 100%|██████████| 10/10 [00:30<00:00,  3.01s/it]

--------------------------------------------- Result 10 ---------------------------------------------
[[Positive (100%)]] --> [[Negative (99%)]]

are more [[deeply]] thought through than in most ` right-thinking ' films 

are more [[bitterly]] thought through than in most ` right-thinking ' films 



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 6      |
| Number of failed attacks:     | 4      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 40.0%  |
| Attack success rate:          | 60.0%  |
| Average perturbed word %:     | 23.3%  |
| Average num. words per input: | 9.5    |
| Avg num queries:              | 86.4   |
+-------------------------------+--------+
